In [1]:
# 합성곱/풀링 계층 구현하기
# 트릭을 활용해 구현한다.

In [5]:
# 4차원 배열

import numpy as np

x = np.random.rand(10, 1, 28, 28) # 높이,너비가 28이고 채널이 1개인 데이터가 10개
print(x.shape)

print(x[0].shape) # 1번째 데이터의 형상
print(x[1].shape) # 2번째 데이터의 형상

print(x[0,0]) # =x[0][0], 1번째 데이터의 첫 채널의 공간 데이터(2차원)

(10, 1, 28, 28)
(1, 28, 28)
(1, 28, 28)
[[0.35928636 0.49746958 0.87204599 0.60713155 0.67984181 0.7019934
  0.27935666 0.20218854 0.08479381 0.74565665 0.61244379 0.25024374
  0.8586138  0.48784567 0.96917434 0.3826998  0.44421049 0.98278279
  0.2491392  0.43099669 0.27004835 0.22591263 0.6433452  0.72634808
  0.44209513 0.22982548 0.63110349 0.58236004]
 [0.09293883 0.91465855 0.13753856 0.62560176 0.46283262 0.99622175
  0.0120011  0.39384064 0.3747254  0.56080016 0.29863784 0.92949047
  0.62947079 0.73488466 0.94457662 0.76456392 0.16547258 0.27269676
  0.59584676 0.56523193 0.24383644 0.0429029  0.59819089 0.93500853
  0.84280392 0.44566552 0.49754628 0.97878253]
 [0.86705388 0.00397301 0.97684001 0.64094978 0.85975467 0.60761511
  0.18077379 0.06095594 0.25138748 0.16236922 0.63731736 0.48387196
  0.97811253 0.88914506 0.68953767 0.58481651 0.22672327 0.42829464
  0.10953394 0.43716207 0.40774468 0.48753857 0.57414029 0.59637136
  0.86727593 0.15576403 0.19353661 0.34904721]
 [0.

In [6]:
# 합성곱 연산을 구현하기위해, for문을 사용하지 않고 im2col이라는 편의 함수를 사용해 간단히 구현한다.
# numpy에서는 원소에 접근할 때 for문을 사용하지 않는 것이 성능에 도움이 된다.
# im2col은 입력 데이터를 필터링(가중치 계산)하기 좋게 2차원으로 전개하는(펼치는) 함수이다.

# im2col을 사용해 구현하면 필터 적용 영역이 겹쳐서 전개한 후의 블록 수가 원래 블록의 원소 수보다 많아질 수 있다.
# 따라서 메모리를 더 많이 소비하지만, 선형 대수 라이브러리를 활용하므로 계산 효율은 높은 편이다.

In [8]:
# 이 책에서는 im2col 함수를 미리 만들어 제공한다.
# im2col(input_data, filter_h, filter_w, stride=1, pad=0) - 이 때 input_data는 (데이터 수, 채널 수, 높이, 너비)의 4차원 배열로 이뤄진 입력 데이터
# 필터 크기, 스트라이드, 패딩을 고려하여 입력 데이터를 2차원 배열로 전개한다.

import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) # (데이터 수(=배치크기), 채널 수, 높이, 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0) 
print(col1.shape) # (9,75) --> 75 = 3(채널수) * 5(필터 높이) * 5(필터 너비)

x2 = np.random.rand(10, 3, 7, 7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) # (90, 75) --> 90 = 9 * 10(데이터수,배치크기)

(9, 75)
(90, 75)


In [9]:
# 합성곱 계층 구현

class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    # 순전파
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 필터 전개 : 각 필터 블록은 1줄로 펼쳐 세운다. 이 때, reshape는 FN*C*FW*FH개의 원소를 FN개의 묶음으로, 즉 (FN,C*FW*FH) 형태로 깔끔하게 묶어준다.
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) # transpose로 축 변경, (0,1,2,3)->(0,3,1,2) == (N,H,W,C)->(N,C,H,W)
        
        return out
    
    # 역전파 - Affine 계층과 똑같은 방식
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN) # transpose로 축 변경, (0,1,2,3)->(0,2,3,1) == (N,C,H,W)->(N,H,W,C), reshape로 FN개 묶음으로 배치

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad) # 주의! im2col이 아닌 col2im 사용

        return dx

In [10]:
# 풀링의 경우 각 적용영역(2차원)을 1차원으로 펼치고, 각 행에서 가장 큰 원소를 골라 (n*1)행렬로 만든뒤 2차원으로 reshape한다
# 풀링 계층 구현

class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, w):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 전개 (1) - 입력 데이터 전개
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w) 
        
        # 최댓값 (2) - 행별 최댓값 구하기
        out = np.max(col, axis = 1) # np.max(x, axis=n) --> 입력 x의 n번째 축마다 최댓값을 구하여 반환. col과 같은 2차원 배열에서, axis=0은 열, axis=1은 행
        
        # 성형 (3) - 출력에 적절한 모습으로 성형
        out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)
        
        return out
    
    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)
        
        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,)) 
        
        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
        
        return dx